# Boosting Trees

Chapter 10. Boosting and Additive Trees

本章节先介绍了 Boosting 方法，经典的 AdaBoost.M1 算法；然后作者把它与 Tree 相结合，卖自己的 GTBA 算法，甚至把 Boosting 捧上了天：
> AdaBoost with trees as the "best off-the-shelf classifier in the world"




## Outline

1. *Boosting* ，用一些弱分类器 (weak classifier) $G_m(x)$，加权迭代操作，最终实现优异的二元分类效果。
\begin{align}
G(x) = sign\left(\sum_{m=1}^M \alpha_mG_m(x)\right).
\end{align}
可以有 Adaboost.M1 算法实现。

1. *Additive model* Boosting 和 Additive model 在原理上是相通的
\begin{align}
f(x) = \sum_{m=1}^M \beta_m b(x;\gamma_m)
\end{align}
在最小化损失函数时，提出向前分布算法 (Forward Stagewise Additive Modeling)
\begin{align}
\min_{\{\beta_m,\gamma_m\}_1^M} \sum_{i=1}^N L \left(y_i, \sum_{m=1}^M \beta_m b(x_i;\gamma_m)  \right).
\end{align}

1. *Boosting Trees* 就是把树
\begin{align}
T(x; \Theta) = & \sum_{j=1}^J \gamma_j I(x \in R_j) \\
\Theta = & \{ R_j, \gamma_j \}_1^J
\end{align}
带入上上述的 Additive model 中
\begin{align}
f_M(x) = \sum_{m=1}^M T(x; \Theta)
\end{align}
注意，上述公式每个数前面没有权重，因为权重已经隐藏在了 $\Theta$ 中。

1. *Gradient Tree Boosting Algorithm* 梯度回归数提升算法，用于解上述每个 Boosting Tree。关于该算法的使用：
  - $4 \leq J \leq 8$ 就够了。
  - Shrinkage ,$v$, 可以降低错误率。
  - 在 Shrinkage 基础上，Subsampling ,$\eta$, 可以进一步提高性能。 



## 具体内容

### AdaBoosting.M1.

若分类器的定义：
> A weak classifier is one whose error rate is only slightly better than random guessing

Boosting 把握住了弱分类器只对部分数据有效，那么把所有的弱分类器 $G_m(x)$ 叠加，若可以覆盖所有数据，那就成功了。先给所有训练数据一些初始权重 $w_i = 1/N $，在每个弱分类器后，成功则降低权重，错误则增加权重，以期待被后面的弱分类器成功分类.
\begin{align}
err_m =& \frac{\sum_{i=1}^N w_i I(y_i \neq G_m(x_i))}{\sum_{i=1}^Nw_i} \\
\alpha_m = & \log((1-err_m)/err_m)\\
w_i \leftarrow & w_i \cdot exp[\alpha_m \cdot I(y_i \neq G_m(x_i))]
\end{align}

### Forward Stagewise Additive Modeling

也是计算每个基函数的最优化参数，然后叠加。
\begin{align}
(\beta_m, \gamma_m) = & \arg \min_{\beta, \gamma} \sum_{i=1}^N L(y_i, f_{m-1}(x_i) + \beta b(x_i;\gamma)) \\
f_m(x) = & f_{m-1}(x) + \beta_m b(x; \gamma_m)
\end{align}

### Forward Stagewise Boosted Tree Modeling

\begin{align}
\hat{\Theta}_m = \arg \min_{\Theta_m}\sum_{i=1}^N L(y_i, f_{m-1}(x_i)+T(x_i;\Theta_m))
\end{align}
通常直接求解两个优化变量非常困难，可以分部进行，反正模型也不完全需要最优：
1. Finding $\gamma_{jm}$ given $R_{jm}$.
1. Finding $R_{jm}$

可以对比上面的 Forward Stagewise Additive Modeling，同样是两个优化参数。


### Gradient Tree Boosting Algorithm

先看传统的梯度下降法，对于目标函数
\begin{align}
L(f) = \sum_{i=1}^N L(y_i,f(x_i))
\end{align}
每次沿着梯度方向递减
\begin{align}
g_{im} =& \left[ \frac{\partial L(y_i, f(x_i))}{\partial f(x_i)}\right]_{f(x_i) = f_{m-1}(x_i)} \\
\rho_m =& \arg \min_{\rho} L(\mathbf{f}_{m-1} -\rho \mathbf{g}_m) \\
\mathbf{f}_m = & \mathbf{f}_{m-1} -\rho_m \mathbf{g}_m
\end{align}

Boosting Tree 有些不同，每次函数的加减必须是一颗树，而不是简单的梯度。这两个的维度不一样，在完全树的情况下，可以等价。但是，Boosting Tree 通常是简单的不完全生成树，所以需要一些近似的操作。方法就是生产一颗新的树 $T_m(x;\Theta)$，使它拟合所有的 梯度 $r_{im}$。
\begin{align}
r_{im} =& -\left[ \frac{\partial L(y_i, f(x_i))}{\partial f(x_i)}\right]_{f(x_i) = f_{m-1}(x_i)} \\
T_m(x;\Theta) \sim  r_{im} \Rightarrow & R_{jm} \\
\gamma_{jm} = & \arg \min_{\gamma} \sum_{x_i \in R_{jm}} L(y_i, f_{m-1}(x_i) + \gamma) \\
f_m(x) =& f_{m-1}(x) + T_m(x;\Theta=(R_{jm},\gamma_{jm}))
\end{align}
